# Using MPT 7b for Airflow Docs Q&A

## Dependencies

In [1]:
%pip install accelerate transformers[torch] torch torchvision einops --user

Note: you may need to restart the kernel to use updated packages.


## GPU Config

In [2]:
import torch

print("GPU Details:", "==="*10, sep="\n")
info = (
    f"Cuda is available?: {torch.cuda.is_available()}\n"
    f"Cuda device count: {torch.cuda.device_count()}\n"
    f"Cuda current device: {torch.cuda.current_device()}\n"
    f"Cuda device id: {torch.cuda.device(0)}\n"
) if torch.cuda.is_available() else (
    f"Cuda is not available!\n"
    f"Check notebook 'llama-cpp.ipynb' to run models on CPU...\n"
)
print(info)

GPU Details:
Cuda is not available!
Running models on CPU...



## Model Setup

In [ ]:
import torch
import transformers

name = 'mosaicml/mpt-7b'

config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
config.attn_config['attn_impl'] = 'triton'
config.init_device = 'cuda:0' # For fast initialization directly on GPU!

model = transformers.AutoModelForCausalLM.from_pretrained(
    name, 
    config=config,
    torch_dtype=torch.bfloat16, # Load model weights in bfloat16
)

Instantiating an MPTForCausalLM model from /home/jovyan/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/a78c1fa391580242211a2f516cf9ef10c86713c8/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Storing model locally
from pathlib import Path
model_path = Path("./models/").mkdir(parents=True, exist_ok=True)
model.save_model(model_path)